In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

In [2]:
!wget https://raw.githubusercontent.com/alexeygrigorev/datasets/master/housing.csv

--2022-09-26 15:01:52--  https://raw.githubusercontent.com/alexeygrigorev/datasets/master/housing.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1423529 (1.4M) [text/plain]
Saving to: ‘housing.csv’

housing.csv         100%[===================>]   1.36M  --.-KB/s    in 0.1s    

2022-09-26 15:01:53 (10.4 MB/s) - ‘housing.csv’ saved [1423529/1423529]



In [3]:
df_housing = pd.read_csv('housing.csv')
df_housing.head(3)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY


In [4]:
df_housing.describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,20640.000000,20640.000000,20640.000000,20640.000000,20433.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,-119.569704,35.631861,28.639486,2635.763081,537.870553,1425.476744,499.539680,3.870671,206855.816909
std,2.003532,2.135952,12.585558,2181.615252,421.385070,1132.462122,382.329753,1.899822,115395.615874
min,-124.350000,32.540000,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,14999.000000
25%,-121.800000,33.930000,18.000000,1447.750000,296.000000,787.000000,280.000000,2.563400,119600.000000
50%,-118.490000,34.260000,29.000000,2127.000000,435.000000,1166.000000,409.000000,3.534800,179700.000000
75%,-118.010000,37.710000,37.000000,3148.000000,647.000000,1725.000000,605.000000,4.743250,264725.000000
max,-114.310000,41.950000,52.000000,39320.000000,6445.000000,35682.000000,6082.000000,15.000100,500001.000000


In [6]:
df_housing.isnull().sum()

longitude               0
latitude                0
housing_median_age      0
total_rooms             0
total_bedrooms        207
population              0
households              0
median_income           0
median_house_value      0
ocean_proximity         0
dtype: int64

## Data preparation

In [7]:
df_housing['total_bedrooms'] = df_housing['total_bedrooms'].fillna(0)

In [9]:
# Create a new column rooms_per_household by dividing the column total_rooms by the column households from dataframe.
df_housing['rooms_per_household'] = df_housing['total_bedrooms'] / df_housing['households'] 
# Create a new column bedrooms_per_room by dividing the column total_bedrooms by the column total_rooms from dataframe.
df_housing['bedrooms_per_room'] = df_housing['total_bedrooms'] / df_housing['total_rooms']
# Create a new column population_per_household by dividing the column population by the column households from dataframe.
df_housing['population_per_household'] = df_housing['population'] / df_housing['households']

In [10]:
df_housing.head(3)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity,rooms_per_household,bedrooms_per_room,population_per_household
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY,1.023810,0.146591,2.555556
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY,0.971880,0.155797,2.109842
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY,1.073446,0.129516,2.802260


### Question 1

In [11]:
df_housing['ocean_proximity'].value_counts()

<1H OCEAN     9136
INLAND        6551
NEAR OCEAN    2658
NEAR BAY      2290
ISLAND           5
Name: ocean_proximity, dtype: int64

### Question 2

In [12]:
average = df_housing['median_house_value'].mean()
df_housing['above_average'] = (df_housing.median_house_value > average).astype(int)

In [17]:
df_full_train , df_test = train_test_split(df_housing, test_size = 0.2, random_state = 42) 
df_train, df_val  = train_test_split(df_full_train, test_size = 0.25, random_state = 42)

df_train = df_train.reset_index(drop = True)
df_val = df_val.reset_index(drop = True)
df_test = df_test.reset_index(drop = True)

y_train = df_train.median_house_value.values
y_val = df_val.median_house_value.values
y_test = df_test.median_house_value.values

del df_train['median_house_value']
del df_val['median_house_value']
del df_test['median_house_value']

In [19]:
df_train.corr().unstack().sort_values(ascending = False).to_frame()

,,0
longitude,longitude,1.000000
latitude,latitude,1.000000
population_per_household,population_per_household,1.000000
bedrooms_per_room,bedrooms_per_room,1.000000
rooms_per_household,rooms_per_household,1.000000
...,...,...
housing_median_age,total_rooms,-0.363522
median_income,bedrooms_per_room,-0.616617
bedrooms_per_room,median_income,-0.616617
longitude,latitude,-0.925005


### Question 3

In [20]:
average = df_full_train['median_house_value'].mean()
df_full_train['above_average'] = (df_full_train.median_house_value > average).astype(int)

In [21]:
from sklearn.metrics import mutual_info_score
round(mutual_info_score(df_full_train.ocean_proximity, df_full_train.above_average), 2)

0.1

### Question 4

In [22]:
y_train = df_train.above_average.values
y_val = df_val.above_average.values
y_test = df_test.above_average.values

del df_train['above_average']
del df_val['above_average']
del df_test['above_average']

In [23]:
num = ['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income',
       'ocean_proximity', 'rooms_per_household', 'bedrooms_per_room',
       'population_per_household']
cat = ['ocean_proximity']

In [24]:
train_dict = df_train[num + cat].to_dict(orient = 'records')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  """Entry point for launching an IPython kernel.


In [26]:
dv = DictVectorizer(sparse = False)
X_train = dv.fit_transform(train_dict)
X_train.shape

(12384, 16)

In [27]:
val_dict = df_val[num + cat].to_dict(orient = 'records')
X_val = dv.transform(val_dict)
model = LogisticRegression(solver="liblinear", C=1.0, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  """Entry point for launching an IPython kernel.


LogisticRegression(max_iter=1000, random_state=42, solver='liblinear')

In [28]:
y_pred = model.predict(X_val)
accuracy = np.round(accuracy_score(y_val, y_pred),2)
accuracy

0.84

### Question 5

In [29]:
features = num + cat

In [30]:
origin_accuracy = accuracy

for c in features:
  sub = features.copy()
  sub.remove(c)

  train_dict = df_train[sub].to_dict(orient = 'records')

  dv = DictVectorizer(sparse = False)
  dv.fit(train_dict)

  X_train  = dv.transform(train_dict)

  model = LogisticRegression(solver="liblinear", C=1.0, max_iter=1000, random_state=42)
  model.fit(X_train, y_train)

  val_dict = df_val[sub].to_dict(orient = 'records')
  X_val = dv.transform(val_dict)
  y_pred = model.predict(X_val)
  score = accuracy_score(y_val, y_pred)

  print(f'Features: {c}, Diff: {origin_accuracy - score}, Score: {score}')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: UserWarning: DataFrame columns are not unique, some columns will be omitted.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  import sys


Features: longitude, Diff: 0.008846899224806193, Score: 0.8311531007751938


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: UserWarning: DataFrame columns are not unique, some columns will be omitted.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  import sys


Features: latitude, Diff: 0.006666666666666599, Score: 0.8333333333333334


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: UserWarning: DataFrame columns are not unique, some columns will be omitted.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  import sys


...

Features: rooms_per_household, Diff: 0.005697674418604581, Score: 0.8343023255813954


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: UserWarning: DataFrame columns are not unique, some columns will be omitted.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  import sys


Features: bedrooms_per_room, Diff: 0.004728682170542564, Score: 0.8352713178294574


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: UserWarning: DataFrame columns are not unique, some columns will be omitted.


Features: population_per_household, Diff: 0.0042441860465115555, Score: 0.8357558139534884
Features: ocean_proximity, Diff: 0.0020639534883720723, Score: 0.8379360465116279


### Question 6

In [32]:
data = df_housing.drop('above_average',axis = 1)

In [33]:
data['median_house_value']=np.log1p(data['median_house_value'])

In [34]:
df_train_full, df_test = train_test_split(data, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_train_full, test_size=0.25, random_state=42)

df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

y_train = df_train.median_house_value.values
y_val = df_val.median_house_value.values
y_test = df_test.median_house_value.values

del df_train['median_house_value']
del df_val['median_house_value']
del df_test['median_house_value']

In [35]:
train_dicts = df_train[num + cat].to_dict(orient ='records')
val_dicts = df_val[num + cat].to_dict(orient ='records')
test_dicts = df_test[num + cat].to_dict(orient ='records')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  This is separate from the ipykernel package so we can avoid doing imports until


In [36]:
dv_r = DictVectorizer(sparse = False)
dv_r.fit(train_dicts)
X_train = dv_r.transform(train_dicts)

dv_r.fit(val_dicts)
X_val = dv_r.transform(val_dicts)

In [40]:
alpha = [0, 0.01, 0.1, 1, 10]

for al in alpha:
  model = Ridge(alpha=al, solver="sag", random_state=42)
  model.fit(X_train, y_train)

  y_pred = model.predict(X_val)
  rmse_score = np.sqrt(mean_squared_error(y_val, y_pred))

  print(f'alpha: {al} rmse_score: {round(rmse_score, 10)}')

alpha: 0 rmse_score: 0.5242046072
alpha: 0.01 rmse_score: 0.5242046072
alpha: 0.1 rmse_score: 0.5242046073
alpha: 1 rmse_score: 0.5242046089
alpha: 10 rmse_score: 0.5242046249
